In [2]:
#The main Jupyter notebook for running our simulation. For now we can also use this notebook to test out code segments

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import CSJModel as csj

## Possible Variables and Parameters in the Model
* Variable: number of Plants in the plot
* Variable for each Plant:
    * base growth rate
    * mean height
    * mean root depth
    * ideal water
    * water tolerance (is this a function of root depth?)
    * % die-off between growing seasons
    * shade tolerance(?)
* Parameter: total carrying capacity of the plot
* Parameter: number of growing seasons
* Parameter: length of growing season
* Variable: Precipitation during growing season
* Output: % of carrying capacity at end of each season

## Possible Interactions to Model
* Competition for resources (space) among plant species
    * Suggest using a single plot carrying capacity that each species participates in filling
* Water availability: a function of precipitation and possibly root depth of plant
* Shade from taller plants helps prevent water loss during drought, but maybe inhibits growth of plants that need full sun(?)

In [2]:
#Model Parameters
length_of_growing_season = 150
n_seasons_per_trial = 10
n_trials = 10000
plot_capacity = 1000
base_soil_sat_loss_rate = 0.05
temp = 20
rel_hum = .7

In [ ]:
class Plot:
    species_list = [] #Holds (Plant, Biomass) Tuples
    capacity = plot_capacity
    soil_saturation = 0.5 #0 <= soil_saturation <= 1.0

class Plant:
    name = 'string'
    plant_type =  #'C3' | 'C4' | 'Forb' | 'Legume'
    root_depth = #'Shallow' | 'Deep'
    plant_height = #'Short' | 'Tall'
    base_growth_rate
    ideal_soil_sat
    ideal_sat_alpha

In [ ]:
def VPD(temp = 20, rel_hum = 0.7):
    return ((0.6108 * np.exp((17.27 * temp)/(temp + 237.3))) - (rel_hum * 0.6108 * np.exp((17.27 * temp)/(temp + 237.3))))